In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import RandomCrop
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger
from torchvision.transforms import Resize, InterpolationMode, ToPILImage
from torchmetrics import JaccardIndex, Precision, Recall, F1Score
import segmentation_models_pytorch as smp

from src.models.BaselineModel import BaselineModel
from src.evaluation.evaluate_result import evaluate_result
from src.callbacks.SaveRandomImagesCallback import SaveRandomImagesCallback
from src.callbacks.SaveTestPreds import SaveTestPreds
from src.datasets.INRIAAerialImageLabellingDataset import (
    INRIAAerialImageLabellingDataset,
)
from src.utils import view_and_save_images_shapes

d:\__repos\aerial_segmentation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare environment

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if you get some cryptic CUDA error, set device to "cpu" and try again
print(device)

cuda


In [4]:
VAL_SIZE = 0.2
BATCH_SIZE = 4
SEED = 42
IMAGE_SIZE = 576
SAVE_VAL_DIR = "outputs/INRIA/val"
SAVE_TEST_DIR = "outputs/INRIA/test"
INRIA_DATASET_PATH = "data/INRIAAerialImageLabellingDataset"  # home PC
# INRIA_DATASET_PATH = "data/TestSubsets/INRIAAerialImageLabellingDataset"  # laptop

In [5]:
labeled_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH,
    split="train",
    transforms=[
        # Resize(IMAGE_SIZE),
        RandomCrop((IMAGE_SIZE, IMAGE_SIZE)),
        # Resize(IMAGE_SIZE, interpolation=InterpolationMode.NEAREST_EXACT),
        # ResizeToDivisibleBy32()
    ],
)
print(len(labeled_dataset))

180


In [6]:
test_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH,
    split="test",
    transforms=[
        # Resize(IMAGE_SIZE),
        RandomCrop((IMAGE_SIZE, IMAGE_SIZE)),
        # Resize(IMAGE_SIZE, interpolation=InterpolationMode.NEAREST_EXACT),
        # ResizeToDivisibleBy32()
    ],
)
print(len(test_dataset))

144


## Prepare data

In [7]:
sanity_check_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH,
    split="train",
)
print(len(sanity_check_dataset))

180


In [8]:
sanity_check_dataloader = DataLoader(
    sanity_check_dataset, batch_size=BATCH_SIZE, shuffle=True
)

In [9]:
view_and_save_images_shapes(sanity_check_dataloader, 'inria_images_shapes', verbose=True)

Opening file inria_images_shapes
Saving image shapes to CSV file... <_io.TextIOWrapper name='inria_images_shapes.csv' mode='a' encoding='cp1250'>
Header written to CSV file... <_io.TextIOWrapper name='inria_images_shapes.csv' mode='a' encoding='cp1250'>
Batch Index: 0
Image Shape: torch.Size([4, 3, 5000, 5000])
Mask Shape: torch.Size([4, 1, 5000, 5000])
Writting rov 0
Batch Index: 1
Image Shape: torch.Size([4, 3, 5000, 5000])
Mask Shape: torch.Size([4, 1, 5000, 5000])
Writting rov 1
Batch Index: 2
Image Shape: torch.Size([4, 3, 5000, 5000])
Mask Shape: torch.Size([4, 1, 5000, 5000])
Writting rov 2
Batch Index: 3
Image Shape: torch.Size([4, 3, 5000, 5000])
Mask Shape: torch.Size([4, 1, 5000, 5000])
Writting rov 3
Batch Index: 4
Image Shape: torch.Size([4, 3, 5000, 5000])
Mask Shape: torch.Size([4, 1, 5000, 5000])
Writting rov 4
Batch Index: 5
Image Shape: torch.Size([4, 3, 5000, 5000])
Mask Shape: torch.Size([4, 1, 5000, 5000])
Writting rov 5
Batch Index: 6
Image Shape: torch.Size([4, 3

### Train, validation and test data

In [ ]:
train_size = int(0.8 * len(labeled_dataset))
val_size = len(labeled_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    labeled_dataset, [train_size, val_size]
)

In [ ]:
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, persistent_workers=True
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, persistent_workers=True
)
test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, persistent_workers=True
)

In [ ]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

In [ ]:
to_pil_transform = ToPILImage()

In [ ]:
if BATCH_SIZE == 1:
    img = to_pil_transform(images.squeeze())

In [ ]:
# img.show()

In [ ]:
if BATCH_SIZE == 1:
    msk = to_pil_transform(masks.squeeze())

In [ ]:
# msk.show()

# Training module

In [ ]:
class SegmentationModel(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super(SegmentationModel, self).__init__()
        
        self.model = model
        self.learning_rate = learning_rate

        # self.criterion = smp.losses.JaccardLoss(mode="binary", from_logits=True)
        self.criterion = smp.losses.MCCLoss()
        self.train_iou = JaccardIndex(num_classes=2, task="binary")
        self.val_iou = JaccardIndex(num_classes=2, task="binary")

        # self.save_hyperparameters()

    def forward(self, x):
        output = self.model(x.to(device))
        return output

    def training_step(self, batch, batch_idx):
        images, masks = batch
        masks = torch.div(masks, 255).float()
        preds = self(images)
        loss = self.criterion(preds, masks)

        self.log("train_loss", loss, on_epoch=True, on_step=True)
        self.log("train_iou", self.train_iou(preds, masks), on_epoch=True, on_step=True)

        return loss

    def validation_step(self, batch, batch_idx):
        images, masks = batch
        masks = torch.div(masks, 255).float()
        preds = self(images)
        loss = self.criterion(preds, masks)

        self.log("val_loss", loss, on_epoch=True, on_step=True)
        self.log("val_iou", self.val_iou(preds, masks), on_epoch=True, on_step=True)

        return loss

    def test_step(self, batch, batch_idx):
        # just here to activate the test_epoch_end
        # callback SaveTestPreds starts on_test_epoch_end 
        pass

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,  # model output channels (number of classes in your dataset)
    activation="sigmoid",
).to(device)

In [ ]:
# model = smp.UnetPlusPlus(
#     encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,  # model output channels (number of classes in your dataset)
# ).to(device)

In [ ]:
# model = smp.DeepLabV3(
#     encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,
# ).to(device)

In [ ]:
# model = smp.DeepLabV3Plus(
#     encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,
# ).to(device)

In [ ]:
segmentation_model = SegmentationModel(model)

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    monitor="val_loss", save_top_k=-1, mode="min"
)

In [ ]:
logger = CSVLogger("lightning_logs", name="inria_segmentation_model")

In [ ]:
save_callback = SaveRandomImagesCallback(save_dir=SAVE_VAL_DIR)
save_test_preds_callback = SaveTestPreds(save_dir=SAVE_TEST_DIR)

In [ ]:
early_stopping_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5)

In [ ]:
trainer = pl.Trainer(
    max_epochs=30,
    log_every_n_steps=10,
    callbacks=[
        model_checkpoint_callback,
        save_callback,
        save_test_preds_callback,
        early_stopping_callback,
    ],
    logger=logger,
)

# DEBUG
# trainer = pl.Trainer(
#     max_epochs=1,
#     callbacks=[model_checkpoint_callback, save_callback],
#     logger=logger,
#     accelerator="cpu"
#     )

In [ ]:
trainer.fit(
    segmentation_model, train_dataloaders=train_loader, val_dataloaders=val_loader
)

In [ ]:
trainer.test(ckpt_path="best", dataloaders=test_loader)